In [3]:
! export JAX_PLATFORMS=cpu

/gpfs/cfel/group/cmi/common/psi4/psi4conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


# Hyperfine solutions and spin-density for H2S

Compute the hyperfine energies and wavefunctions for selected rotational cluster states of $\text{H}_2\text{S}$.
The hyperfine interaction is represented by the spin-rotation coupling.

We also compute the nuclear spin densities to verify if the ortho-para interaction can induce the nuclear spin polarisation.

In [4]:
import h5py
import hdfdict
import jax
import numpy as np
from jax import config
from jax import numpy as jnp
from rovib.cartens import CART_IND
from rovib.spinrot_xy2 import spinrot_xy2
from rovib.symtop import threej_wang
from scipy import constants
from scipy.sparse import csr_matrix

config.update("jax_enable_x64", True)
KHZ_TO_INVCM = 1.0 / constants.value("speed of light in vacuum") * 10

First, read the indices of rotational cluster states for different $J$ quanta, sored in file `h2s_cluster_states.txt`. The rotational cluster states were identified in `h2s_clusters.ipynb`.

In [5]:
cluster_state_id = {}
with open("h2s_cluster_states_id.txt", "r") as fl:
    for line in fl:
        w = line.split()
        j = [int(w[0 + i * 12]) for i in range(4)]
        id = [int(w[1 + i * 12]) for i in range(4)]
        sym = [w[3 + i * 12] for i in range(4)]
        assert all(j[0] == elem for elem in j), f"Not all J quanta are equal: {j}"
        j_ = j[0]
        if j_ >= 50:
            cluster_state_id[j_] = {}
            for sym_, id_ in zip(sym, id):
                try:
                    cluster_state_id[j_][sym_].append(id_)
                except KeyError:
                    cluster_state_id[j_][sym_] = [id_]
            print(f"J = {j_}, cluster-state IDs: {cluster_state_id[j_]}")

J = 50, cluster-state IDs: {'B2': [277], 'A2': [242], 'B1': [242], 'A1': [280]}
J = 51, cluster-state IDs: {'A1': [267], 'B1': [299], 'B2': [270], 'A2': [299]}
J = 52, cluster-state IDs: {'B2': [324], 'A2': [291], 'A1': [327], 'B1': [291]}
J = 53, cluster-state IDs: {'A1': [316], 'B1': [354], 'B2': [318], 'A2': [353]}
J = 54, cluster-state IDs: {'B2': [385], 'A2': [345], 'A1': [387], 'B1': [344]}
J = 55, cluster-state IDs: {'A1': [371], 'B1': [413], 'A2': [412], 'B2': [373]}
J = 56, cluster-state IDs: {'B2': [447], 'A2': [399], 'B1': [399], 'A1': [450]}
J = 57, cluster-state IDs: {'A1': [432], 'B1': [487], 'B2': [434], 'A2': [486]}
J = 58, cluster-state IDs: {'B2': [520], 'B1': [468], 'A1': [523], 'A2': [469]}
J = 59, cluster-state IDs: {'A1': [503], 'B1': [565], 'A2': [563], 'B2': [505]}
J = 60, cluster-state IDs: {'B1': [542], 'A1': [600], 'A2': [544], 'B2': [599]}


Read energies and spin-rotation matrix elemetns for selected (cluster) states.

In [6]:
pmax = 20
min_J = 50
max_J = 50

with h5py.File(f"h2s_spinrot_enr_pmax{pmax}.h5", "r") as h5:
    enr = hdfdict.load(h5.require_group("energies"), False)

enr_cluster = {
    int(J): {
        sym: enr_sym[cluster_state_id[int(J)][sym]] for sym, enr_sym in enr_J.items()
    }
    for J, enr_J in enr.items()
}

sr1_cluster = {}
sr2_cluster = {}
for J1 in range(min_J, max_J + 1):
    for J2 in range(min_J, max_J + 1):
        if abs(J1 - J2) > 2:
            continue
        with h5py.File(f"h2s_spinrot_me_pmax{pmax}_j{J1}_j{J2}.h5", "r") as h5:
            sr1 = hdfdict.load(h5.require_group("h1"), False)
            sr2 = hdfdict.load(h5.require_group("h2"), False)

        for i, sr in enumerate((sr1, sr2)):
            sr_ = {
                (sym1, sym2): csr_matrix(
                    (me["data"], me["indices"], me["indptr"]), shape=me["shape"]
                )
                .toarray()
                .reshape(len(enr[str(J1)][sym1]), len(enr[str(J2)][sym2]), 3)[
                    np.ix_(cluster_state_id[J1][sym1], cluster_state_id[J2][sym2])
                ]
                for sym1, me_sym in sr.items()
                for sym2, me in me_sym.items()
            }
            if i == 0:
                sr1_cluster[(J1, J2)] = sr_
            else:
                sr2_cluster[(J1, J2)] = sr_

Compute hyperfine energies and wavefunctions

In [7]:
F = 50
J = F

hyper_enr, hyper_vec, hyper_qua = spinrot_xy2(
    F,
    {J: enr_cluster[J]},
    {(J, J): sr1_cluster[(J, J)]},
    {(J, J): sr2_cluster[(J, J)]},
    # enr_cluster,
    # sr1_cluster,
    # sr2_cluster,
    spin_states=((0, "B2"), (1, "A1")),
    spins=(0.5, 0.5),
)

for sym in hyper_enr.keys():
    ind = np.argmax(hyper_vec[sym] ** 2, axis=0)
    for ii, (e, i) in enumerate(zip(hyper_enr[sym], ind)):
        qua = hyper_qua[sym][i]
        rov_sym = qua[1]
        print(
            sym,
            e,
            hyper_qua[sym][i],
            (e - enr_cluster[J][rov_sym]) / KHZ_TO_INVCM,
            hyper_vec[sym][i, ii],
        )


# for key, val in hyper_enr.items():
#     print(key, val.shape, val)
# print(enr[J])

B1 24708.81878741142 (50, 'A2', 0, 'B2') [-251.31312801] (-0.9645884203858239-0j)
B1 24708.81890790879 (50, 'B1', 1, 'A1') [235.51446432] (0.9645884203858239+0j)
B2 24708.818786957007 (50, 'B2', 1, 'A1') [-264.91295521] (-0.9651734896744998+0j)
B2 24708.818908370573 (50, 'A1', 0, 'B2') [249.11429151] (-0.9651734896744998-0j)


In [12]:
with h5py.File("h2s_cluster_states_me.h5", "r") as h5:
    enr = hdfdict.load(h5.require_group("energies"), False)
    sr = hdfdict.load(h5.require_group("spin-rotation"), False)

enr = {
    int(J): {sym: enr_sym for sym, enr_sym in enr_J.items()} for J, enr_J in enr.items()
}

sr_h1 = {
    (int(J), int(J)): {
        (sym1, sym2): me_sym2
        for sym1, me_sym1 in me_J.items()
        for sym2, me_sym2 in me_sym1.items()
    }
    for J, me_J in sr["h1"].items()
}

sr_h2 = {
    (int(J), int(J)): {
        (sym1, sym2): me_sym2
        for sym1, me_sym1 in me_J.items()
        for sym2, me_sym2 in me_sym1.items()
    }
    for J, me_J in sr["h2"].items()
}

In [13]:
F = 50
J = F

hyper_enr, hyper_vec, hyper_qua = spinrot_xy2(
    F,
    {J: enr[J]},
    {(J, J): sr_h1[(J, J)]},
    {(J, J): sr_h2[(J, J)]},
    spin_states=((0, "B2"), (1, "A1")),
    spins=(0.5, 0.5),
)

for sym in hyper_enr.keys():
    ind = np.argmax(hyper_vec[sym] ** 2, axis=0)
    for ii, (e, i) in enumerate(zip(hyper_enr[sym], ind)):
        qua = hyper_qua[sym][i]
        rov_sym = qua[1]
        print(
            sym,
            e,
            hyper_qua[sym][i],
            (e - enr[J][rov_sym]) / KHZ_TO_INVCM,
            hyper_vec[sym][i, ii],
        )

print(hyper_enr["B1"])
print(enr[J]["B1"], enr[J]["A2"])

# for key, val in hyper_enr.items():
#     print(key, val.shape, val)
# print(enr[J])

B1 24708.81878741142 (50, 'A2', 0, 'B2') [-251.31312801] (-0.9645884203858242-0j)
B1 24708.81890790879 (50, 'B1', 1, 'A1') [235.51446432] (0.9645884203858242+0j)
B2 24708.818786957007 (50, 'B2', 1, 'A1') [-264.91295521] (-0.9651734896745006+0j)
B2 24708.818908370573 (50, 'A1', 0, 'B2') [249.11429151] (-0.9651734896745006-0j)
[24708.81878741 24708.81890791]
[24708.81890005] [24708.81879579]


In [14]:
J = 50
for sym in sr_h1[(J, J)].keys():
    print(sym, np.max(np.abs(sr_h1[(J, J)][sym] - sr1_cluster[(J, J)][sym])))

for sym in sr_h1[(J, J)].keys():
    print(sym, sr_h1[(J, J)][sym], sr1_cluster[(J, J)][sym])

('A1', 'A1') 4.618527782440651e-14
('A1', 'B2') 3.503224013374773e-08
('A2', 'A2') 5.595524044110789e-14
('A2', 'B1') 3.5029605616438325e-08
('B1', 'A2') 3.502960560754532e-08
('B1', 'B1') 4.884981308350689e-14
('B2', 'A1') 3.503224014133214e-08
('B2', 'B2') 7.238654120556021e-14
('A1', 'A1') [[[-2.73188857+0.j  0.        +0.j -0.00641101+0.j]]] [[[-2.73188857+0.j  0.        +0.j -0.00641101+0.j]]]
('A1', 'B2') [[[ 0.00000000e+00+0.j  3.50322401e-08+0.j -2.21833679e+00+0.j]]] [[[ 0.        +0.j  0.        +0.j -2.21833679+0.j]]]
('A2', 'A2') [[[-2.73188857+0.j  0.        +0.j -0.0064111 +0.j]]] [[[-2.73188857+0.j  0.        +0.j -0.0064111 +0.j]]]
('A2', 'B1') [[[0.00000000e+00+0.j 3.50296056e-08+0.j 2.21833679e+00+0.j]]] [[[0.        +0.j 0.        +0.j 2.21833679+0.j]]]
('B1', 'A2') [[[ 0.00000000e+00+0.j -3.50296056e-08+0.j  2.21833679e+00+0.j]]] [[[0.        +0.j 0.        +0.j 2.21833679+0.j]]]
('B1', 'B1') [[[-2.73188857+0.j  0.        +0.j -0.00641101+0.j]]] [[[-2.73188857+0.j  

A1 A1 [[[-2.73188857+0.j  0.        +0.j -0.00641101+0.j]]]
A1 B2 [[[ 0.00000000e+00+0.j  3.50322401e-08+0.j -2.21833679e+00+0.j]]]
A2 A2 [[[-2.73188857+0.j  0.        +0.j -0.0064111 +0.j]]]
A2 B1 [[[0.00000000e+00+0.j 3.50296056e-08+0.j 2.21833679e+00+0.j]]]
B1 A2 [[[ 0.00000000e+00+0.j -3.50296056e-08+0.j  2.21833679e+00+0.j]]]
B1 B1 [[[-2.73188857+0.j  0.        +0.j -0.00641101+0.j]]]
B2 A1 [[[ 0.00000000e+00+0.j -3.50322401e-08+0.j -2.21833679e+00+0.j]]]
B2 B2 [[[-2.73188857+0.j  0.        +0.j -0.0064111 +0.j]]]

In [13]:
for sym in enr[50].keys():
    print(sym, enr[50][sym], enr_cluster[50][sym])

A1 [24708.81890006] [24708.81890006]
A2 [24708.81879579] [24708.81879579]
B1 [24708.81890005] [24708.81890005]
B2 [24708.81879579] [24708.81879579]
